In [4]:
import numpy as np
import cv2

In [ ]:
## haar cascade (고양이 얼굴 검출기)

cat_face_cascade = cv2.CascadeClassifier('haarcascade_frontalcatface_extended.xml')

img = cv2.imread('image/cats.jpg', 1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cat_faces = cat_face_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in cat_faces :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## haar cascade (눈, 얼굴 검출기)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

img = cv2.imread('image/aespa.jpg', 1)
img = cv2.resize(img, dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 3)
eyes = eye_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in faces :
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
for (x,y,w,h) in eyes :
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)

    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## ORB 알고리즘(Oriented fast and Rotated Brief)

src = cv2.imread('image/apple_book.jpg', 1)
##src = cv2.resize(src, dsize = (0,0), fx = 0.5, fy = 0.5)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

target = cv2.imread('image/apple.jpg', 0)
##target = cv2.resize(target, dsize = (0,0), fx = 0.5, fy = 0.5)

orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(gray, None)
kp2, des2 = orb.detectAndCompute(target, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x: x.distance)

for i in matches[:100]:
    idx_des1 = i.queryIdx
    x1_des1, y1_des1, =  kp1[idx_des1].pt
    
    idx_des2 = i.trainIdx
    x1_des2, y1_des2, =  kp2[idx_des2].pt
    
##    idx = i.imgIdx
##    x1, x2 = kp1[idx].pt
    
    cv2.circle(src, (int(x1_des1), int(y1_des1)), 3, (0, 0, 0), 3)
    cv2.circle(target, (int(x1_des2), int(y1_des2)), 3, (255, 0, 0), 3)
  ##  cv2.circle(src, (int(x1), int(y1)), 3, (255, 255, 255), 3)
    
cv2.imshow('target', target)
cv2.imshow('src', src)
res = cv2.drawMatches(gray, kp1, target, kp2, matches, None, \
                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
cv2.imshow('res', res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## K - means clustering (평균 군집화) // 비지도 학습

src = cv2.imread('image/flower.jpg', 1)

data = src.reshape(-1,3).astype(np.float32)
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)
retval, bestLabels, centers = cv2.kmeans(data, 10, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = centers.astype(np.uint8)
dst = centers[bestLabels].reshape(src.shape)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
print(centers)

In [8]:
## K - nearest neighbor (최근접 이웃) // 지도학습

def loadTrainData(image_path, label_path):
    
    with open(image_path, "rb") as image_data:
        images = np.frombuffer(image_data.read(), dtype = np.uint8, offset = 16)
        
    with open(label_path, "rb") as label_data:
        labels = np.frombuffer(label_data.read(), dtype = np.uint8, offset = 8)
        
    return images.reshape(-1, 784), labels


train_x, train_y = loadTrainData(
    
    "./fashion-mnist/train-images-idx3-ubyte",
    "./fashion-mnist/train-labels-idx1-ubyte"
)

test_x, test_y = loadTrainData(
    
    "./fashion-mnist/t10k-images-idx3-ubyte",
    "./fashion-mnist/t10k-labels-idx1-ubyte"
)


label_dict = {
    
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot",
    
}

knn = cv2.ml.KNearest_create()
retval = knn.train(train_x.astype(np.float32), cv2.ml.ROW_SAMPLE, train_y.astype(np.int32))

count = 500

retval, results, neighborResponses, dist = knn.findNearest(
    test_x[:count].astype(np.float32), k = 13
    
)

matches = results.astype(np.uint8) == test_y[:count][:, None]

print(np.count_nonzero(matches) / count * 100)


for idx, result in enumerate(results):
    print("Index : {}".format(idx))
    print("예측값 : {}".format(label_dict[int(result)]))
    print("싧제값 : {}".format(label_dict[test_y[idx]]))
    cv2.imshow("images", test_x[idx].reshape(28,28,1))
    if cv2.waitKey(0) == ord('q') : 
        break
    else :
        continue

cv2.destroyAllWindows()

86.8
Index : 0
예측값 : Ankle boot
싧제값 : Ankle boot
Index : 1
예측값 : Pullover
싧제값 : Pullover
Index : 2
예측값 : Trouser
싧제값 : Trouser
Index : 3
예측값 : Trouser
싧제값 : Trouser
Index : 4
예측값 : Shirt
싧제값 : Shirt
Index : 5
예측값 : Trouser
싧제값 : Trouser
Index : 6
예측값 : Coat
싧제값 : Coat
Index : 7
예측값 : Shirt
싧제값 : Shirt
Index : 8
예측값 : Sandal
싧제값 : Sandal
Index : 9
예측값 : Sneaker
싧제값 : Sneaker
Index : 10
예측값 : Coat
싧제값 : Coat
Index : 11
예측값 : Ankle boot
싧제값 : Sandal
Index : 12
예측값 : Sandal
싧제값 : Sneaker
Index : 13
예측값 : Dress
싧제값 : Dress
Index : 14
예측값 : Coat
싧제값 : Coat
Index : 15
예측값 : Trouser
싧제값 : Trouser
Index : 16
예측값 : Pullover
싧제값 : Pullover
Index : 17
예측값 : Pullover
싧제값 : Coat
Index : 18
예측값 : Bag
싧제값 : Bag
Index : 19
예측값 : T-shirt/top
싧제값 : T-shirt/top
Index : 20
예측값 : Pullover
싧제값 : Pullover
Index : 21
예측값 : Sneaker
싧제값 : Sandal
Index : 22
예측값 : Sneaker
싧제값 : Sneaker
Index : 23
예측값 : Ankle boot
싧제값 : Ankle boot
Index : 24
예측값 : Trouser
싧제값 : Trouser
Index : 25
예측값 : Pullover
싧제값 : Coat
Index : 2

Index : 210
예측값 : Coat
싧제값 : Coat
Index : 211
예측값 : Trouser
싧제값 : Trouser
Index : 212
예측값 : Bag
싧제값 : Bag
Index : 213
예측값 : Coat
싧제값 : Coat
Index : 214
예측값 : Trouser
싧제값 : Trouser
Index : 215
예측값 : Dress
싧제값 : Dress
Index : 216
예측값 : Trouser
싧제값 : Trouser
Index : 217
예측값 : Shirt
싧제값 : Shirt
Index : 218
예측값 : Sneaker
싧제값 : Sneaker
Index : 219
예측값 : Pullover
싧제값 : Pullover
Index : 220
예측값 : Bag
싧제값 : Bag
Index : 221
예측값 : Sandal
싧제값 : Sandal
Index : 222
예측값 : Pullover
싧제값 : Pullover
Index : 223
예측값 : T-shirt/top
싧제값 : T-shirt/top
Index : 224
예측값 : Sneaker
싧제값 : Sneaker
Index : 225
예측값 : Sneaker
싧제값 : Sneaker
Index : 226
예측값 : Shirt
싧제값 : Shirt
Index : 227
예측값 : Shirt
싧제값 : Pullover
Index : 228
예측값 : Sneaker
싧제값 : Sneaker
Index : 229
예측값 : T-shirt/top
싧제값 : T-shirt/top
Index : 230
예측값 : Sneaker
싧제값 : Sneaker
Index : 231
예측값 : Bag
싧제값 : Bag
Index : 232
예측값 : Ankle boot
싧제값 : Ankle boot
Index : 233
예측값 : Pullover
싧제값 : Pullover
Index : 234
예측값 : Ankle boot
싧제값 : Ankle boot
Index : 235
예측값 :

Index : 418
예측값 : Sandal
싧제값 : Sandal
Index : 419
예측값 : Pullover
싧제값 : Coat
Index : 420
예측값 : Sandal
싧제값 : Sandal
Index : 421
예측값 : Dress
싧제값 : Dress
Index : 422
예측값 : Bag
싧제값 : Bag
Index : 423
예측값 : Sandal
싧제값 : Sandal
Index : 424
예측값 : Bag
싧제값 : Bag
Index : 425
예측값 : Coat
싧제값 : Coat
Index : 426
예측값 : Coat
싧제값 : Coat
Index : 427
예측값 : Bag
싧제값 : Bag
Index : 428
예측값 : Ankle boot
싧제값 : Ankle boot
Index : 429
예측값 : Bag
싧제값 : Bag
Index : 430
예측값 : Shirt
싧제값 : Shirt
Index : 431
예측값 : Pullover
싧제값 : Pullover
Index : 432
예측값 : Coat
싧제값 : Coat
Index : 433
예측값 : Coat
싧제값 : Coat
Index : 434
예측값 : Pullover
싧제값 : Pullover
Index : 435
예측값 : Coat
싧제값 : Coat
Index : 436
예측값 : Trouser
싧제값 : Trouser
Index : 437
예측값 : Shirt
싧제값 : Shirt
Index : 438
예측값 : Trouser
싧제값 : Trouser
Index : 439
예측값 : Dress
싧제값 : Dress
Index : 440
예측값 : T-shirt/top
싧제값 : T-shirt/top
Index : 441
예측값 : Sneaker
싧제값 : Ankle boot
Index : 442
예측값 : Bag
싧제값 : Bag
Index : 443
예측값 : Bag
싧제값 : Bag
Index : 444
예측값 : Coat
싧제값 : Shirt
Index 

In [11]:
print(test_x)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
## optical flow (루카스 - 카나데 알고리즘)

cap = cv2.VideoCapture('image/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

color = (0,255,0)
lines = None
prevImg = None
## calcOpticalFlowPyrLK 중지 요견 설정
termcriteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    img_draw = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prevImg is None:
        prevImg = gray
        
        lines = np.zeros_like(frame)
        
        prevPt = cv2.goodFeaturesToTrack(prevImg, 200, 0.01, 10)
        
    else:
        nextImg = gray
        
        nextPt, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, \
                                                      prevPt, None, \
                                                      criteria=termcriteria)
        
        prevMv = prevPt[status==1]
        nextMv = nextPt[status==1]
        for i,(p, n) in enumerate(zip(prevMv, nextMv)):
            px,py = p.ravel()
            nx,ny = n.ravel()
            
            cv2.line(lines, (px.astype(np.uint16),py.astype(np.uint16)), \
                     (nx.astype(np.uint16),ny.astype(np.uint16)), \
                     (0,255,0), 2)
            
            cv2.circle(img_draw, (nx.astype(np.uint16),ny.astype(np.uint16)),\
                       2, (0,255,0), -1)
            
        img_draw = cv2.add(img_draw,lines)
        
        prevImg = nextImg
        prevPt = nextMv.reshape(-1,1,2)
    
    cv2.imshow('OpticalFlow_LK', img_draw)
    
    if cv2.waitKey(delay) == ord('q'):
        break
    elif cv2.waitKey(delay) == ord('r'):
        prevImg = None

cv2.destroyAllWindows()
cap.release()

In [ ]:
print(prevPt[status==1])
print(prevPt)

In [ ]:
## optical flow (군나르 파너백 알고리즘)

def drawFlow(img, flow, step = 16):
    h,w = img.shape[:2]
    
    idx_y, idx_x = np.mgrid[step/2:h:step,step/2:w:step].astype(np.int)
    indices = np.stack((idx_x, idx_y), axis = -1).reshape(-1,2)
    
    for x,y in indices:
        
        cv2.circle(img, (x,y), 1, (0,255,0), -1)
        
        dx,dy = flow[y, x].astype(np.int)
        
        cv2.line(img, (x,y), (x+dx, y+dy), (0,255,0),2,cv2.LINE_AA)
        
prev = None

cap = cv2.VideoCapture('image/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

while cap.isOpened():
    ret,frame = cap.read()
    if not ret: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev is None:
        prev = gray
    else :
        flow = cv2.calcOpticalFlowFarneback(prev,gray,None,\
                                           0.5,3,15,3,5,1.1,\
                                            cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        drawFlow(frame,flow)
        
        prev = gray
        
    cv2.imshow('op', frame)
    if cv2.waitKey(delay) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

In [ ]:
## 가우시안 블러

img = cv2.imread('image/tomato.jpg', 1)
img = cv2.resize(img, dsize=(0,0), fx = 0.5, fy = 0.5,
                 interpolation = cv2.INTER_AREA)

blur_gaussian = img.copy()
blur = img.copy()

size = (11,11)

blur_gaussian = cv2.GaussianBlur(blur_gaussian, size, sigmaX = 0)
blur = cv2.blur(blur, size)

cv2.imshow('blur',blur)
cv2.imshow('blur_gaussian',blur_gaussian)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()